# Разработка моделей предсказания возникновения сердечно-сосудистых заболеваний
## Данные UCI о болезнях сердца

Каждый день сердце среднестатистического человека бьется около 100 000 раз, перекачивая по телу 7600 литров крови. Длина всех кровеносных сосудов взрослого человека имеет протяжённость приблизительно 100 000 км. Все кровеносные сосуды образуют вместе с сердцем анатомическую основу для системы кровообращения и тем самым, транспорт кислорода и питательных веществ во всё тело.

## Сердечные заболевания

Заболевание сердца описываются - состояние, влияющее на сердце и кровяные сосуды сердца. Это может быть заболевание коронарной артерии, недостаточность сердца или аритмия сердца.

Сердечный приступ может случиться внезапно, а также происходить в течение нескольких часов, дней или даже недель до приступа. Хотя заболевание сердца не имеет симптомов, как таковых, все же должны существовать какие-либо признаки. У мужчин и женщин боль в грудной клетке или руке или дискомфорт могут указывать на заболевание сердца. Боль может быть резкой или умеренной, она может длиться от нескольких минут и возвращаться опять. Также боль может стать потенциальным сигналом для сердечного приступа.


## Описания столбцов:
* id (Уникальный идентификатор для каждого пациента)
* age (возраст пациента в годах)
* origin (место сбора данных)
* sex (мужской/женский)
* cp chest pain type (тип боли в груди - [типичная стенокардия, атипичная стенокардия, неангинальная, бессимптомная])
* trestbps resting blood pressure (артериальное давление в состоянии покоя (в мм рт. ст. при поступлении в больницу))
* chol (холестерин в сыворотке крови в мг/дл)
* fbs (если уровень сахара в крови натощак > 120 мг/дл)
* restecg (результаты электрокардиографии в состоянии покоя - [норма, аномалия stt, гипертрофия lv])
* thalach (максимальная частота сердечных сокращений)
* exang (стенокардия, вызванная физической нагрузкой (правда/ложь))
* oldpeak (Депрессия ST-сегмента, вызванная физической нагрузкой по сравнению с покоем)
* slope (наклон сегмента ST пика физической нагрузки)
* ca (количество крупных сосудов (0-3), окрашенных при рентгеноскопии)
* thal (талассемии  - [норма; фиксированный дефект; обратимый дефект])
* num (прогнозируемый признак - [0=нет сердечных заболеваний; 1,2,3,4 = какое-то сердечное заболевание])

### Загружаем необходимые библиотеки и датасет


In [47]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


In [48]:
# Загружаем данные
data = pd.read_csv('heart_disease_uci.csv')
data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [50]:
print("Количество наблюдений:", data.shape[0])
print("Пропусков в данных:", data.isna().sum().sum())
print("Количество повторяющихся строк:", data.duplicated().sum())

Количество наблюдений: 920
Пропусков в данных: 1759
Количество повторяющихся строк: 0


### Обработка  данных:

Если в записи какого-либо пациента есть недостающие значения, мы полностью удалим эту запись. 

Удалим столбец dataset, который просто информирует нас о месте сбора данных.

In [51]:
data.dropna(inplace=True, axis=0)
data.drop(["id", "dataset"], inplace=True, axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 748
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       299 non-null    int64  
 1   sex       299 non-null    object 
 2   cp        299 non-null    object 
 3   trestbps  299 non-null    float64
 4   chol      299 non-null    float64
 5   fbs       299 non-null    object 
 6   restecg   299 non-null    object 
 7   thalch    299 non-null    float64
 8   exang     299 non-null    object 
 9   oldpeak   299 non-null    float64
 10  slope     299 non-null    object 
 11  ca        299 non-null    float64
 12  thal      299 non-null    object 
 13  num       299 non-null    int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 35.0+ KB


In [52]:
print("Категориальные атрибуты:")
kategorial = data.select_dtypes(include='object').columns.to_list()
kategorial

Категориальные атрибуты:


['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

ВАЖНО! 

sex, fbs, exang бинарные признаки. Необходимо их перекодировать.

In [53]:
data['sex'] = (data['sex'] == 'Male')*1
data['fbs'] = (data['fbs'] == 'True')*1
data['exang'] = (data['exang'] == 'True')*1

Теперь:

In [54]:
print("Категориальные атрибуты:")
kategorial = data.select_dtypes(include='object').columns.to_list()
kategorial

Категориальные атрибуты:


['cp', 'restecg', 'slope', 'thal']

Преобразуем имеющиеся категориальные признаки



In [55]:
for kat in kategorial:
    print("Столбец -", kat, "\n уникальные значения -", data[kat].unique())

Столбец - cp 
 уникальные значения - ['typical angina' 'asymptomatic' 'non-anginal' 'atypical angina']
Столбец - restecg 
 уникальные значения - ['lv hypertrophy' 'normal' 'st-t abnormality']
Столбец - slope 
 уникальные значения - ['downsloping' 'flat' 'upsloping']
Столбец - thal 
 уникальные значения - ['fixed defect' 'normal' 'reversable defect']


In [56]:
data = pd.get_dummies(data)

In [57]:
data

,age,sex,trestbps,chol,fbs,thalch,exang,oldpeak,ca,num,...,cp_typical angina,restecg_lv hypertrophy,restecg_normal,restecg_st-t abnormality,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
0,63,1,145.0,233.0,0,150.0,0,2.3,0.0,0,...,1,1,0,0,1,0,0,1,0,0
1,67,1,160.0,286.0,0,108.0,0,1.5,3.0,2,...,0,1,0,0,0,1,0,0,1,0
2,67,1,120.0,229.0,0,129.0,0,2.6,2.0,1,...,0,1,0,0,0,1,0,0,0,1
3,37,1,130.0,250.0,0,187.0,0,3.5,0.0,0,...,0,0,1,0,1,0,0,0,1,0
4,41,0,130.0,204.0,0,172.0,0,1.4,0.0,0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,68,1,144.0,193.0,0,141.0,0,3.4,2.0,2,...,0,0,1,0,0,1,0,0,0,1
300,57,1,130.0,131.0,0,115.0,0,1.2,1.0,3,...,0,0,1,0,0,1,0,0,0,1
301,57,0,130.0,236.0,0,174.0,0,0.0,1.0,1,...,0,1,0,0,0,1,0,0,1,0
508,47,1,150.0,226.0,0,98.0,0,1.5,0.0,1,...,0,0,1,0,0,1,0,0,0,1


Будем рассматривать задачу "предсказание возникновения сердечно-сосудистых заболеваний" как задачу бинарной классификации, т.е. 1 - у пациента полезнь сердца, 0 - нет сердечных заболеваний.

In [58]:
# Уникальные значения для num
data['num'].unique()

array([0, 2, 1, 3, 4], dtype=int64)

In [59]:
data['num'].replace({2: 1, 3: 1, 4: 1}, inplace =True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 748
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    int64  
 1   sex                       299 non-null    int32  
 2   trestbps                  299 non-null    float64
 3   chol                      299 non-null    float64
 4   fbs                       299 non-null    int32  
 5   thalch                    299 non-null    float64
 6   exang                     299 non-null    int32  
 7   oldpeak                   299 non-null    float64
 8   ca                        299 non-null    float64
 9   num                       299 non-null    int64  
 10  cp_asymptomatic           299 non-null    uint8  
 11  cp_atypical angina        299 non-null    uint8  
 12  cp_non-anginal            299 non-null    uint8  
 13  cp_typical angina         299 non-null    uint8  
 14  restecg_lv

### Подготовка данных для обучения моделей

In [61]:
# Признаки
X = data.drop(["num"], axis=1)
# Ответы
y = data["num"]